This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
#API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [40]:
# First, import the relevant modules
import requests


In [52]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
params=dict(key=API_KEY,start_date='2019-02-01',end_date='2019-02-01')
res=requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json',params)


res.text

'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2019-02-01","end_date":"2019-02-01","frequency":"daily","data":[["2019-02-01",null,81.65,78.2,78.45,null,null,null,null,null,null]],"collapse":null,"order":null}}'

In [53]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(res.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2019-02-01', 'end_date': '2019-02-01', 'frequency': 'daily', 'data': [['2019-02-01', None, 81.65, 78.2, 78.45, None, None, None, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [54]:
#1Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
params=dict(key='API_KEY',start_date='2017-01-01',end_date='2017-12-31')
data=requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json',params)


In [55]:
#2Convert the returned JSON object into a Python dictionary.
import json 
  
dict2=data.json()

print(type(dict2))

<class 'dict'>


In [70]:
#3Calculate what the highest and lowest opening prices were for the stock in this period.
omax=0
omin=float("inf")
for row in dict2['dataset_data']['data']:
    if row[1] != None:
        if row[1]>omax:
            omax=row[1]
        if row[1]<omin:
            omin=row[1]

print("Highest opening price in 2017 was ${0:.2f} and lowest opening price in 2017 was ${1:.2f}." \
      .format(round(omax, 2), round(omin, 2)))   

Highest opening price in 2017 was $53.11 and lowest opening price in 2017 was $34.00.


In [72]:
#4What was the largest change in any one day (based on High and Low price)
dmax=0
for row in dict2['dataset_data']['data']:
    if row[1]!=None:
        if row[2]-row[3]> dmax:
            dmax=row[2]-row[3]
print("Largest change in one day in 2017 was ${0:.2f}." .format(round(dmax,2)))        

Largest change in one day in 2017 was $2.81.


In [79]:
#5What was the largest change between any two days (based on Closing Price)?
#tmax=0
#tmin=float('inf')
#for row in dict2['dataset_data']['data']:
#    if row[4]!=None:
 #       if row[4]>tmax:
  #          tmax=row[4]
   #     if row[4]<tmin:
    #        tmin= row[4]
#print('largest change between any two days in 2017 was ${0:.2f}.'.format(round((tmax-tmin),2)))            

largest change between any two days in 2017 was $19.03.


In [88]:
#5What was the largest change between any two continous days (based on Closing Price)?
day_diff_max = {}
close_early =  dict2["dataset_data"]["data"][0][4]

for row in dict2["dataset_data"]["data"][1:]:
    close = row[4]
    day_diff_max[row[0]] = abs(close_early - close)
    close_early = close

day_diff_max_date, day_diff_max_value = max(day_diff_max.items(), key=lambda k:k[1])
print("The largest change between any two days (based on Closing Price) was ${0:.2f} on {1} ." \
      .format(round(day_diff_max_value, 2), (day_diff_max_date)))

The largest change between any two days (based on Closing Price) was $2.56 on 2017-08-08 .


In [89]:
#6 What was the average daily trading volume during this year?
trad_volu=[row[6] for row in dict2['dataset_data']['data']]
avg=sum(trad_volu)/len(trad_volu)

print('the average daily trading volume during this year was ${0:.2f}.'.format(round(avg, 2)))   
    

the average daily trading volume during this year was $89124.34.


In [83]:
#7 (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
trd_vol= [row[6] for row in dict2['dataset_data']['data']]
sortedtrd = sorted(trd_vol) # in ascending order
n=len(sortedtrd)
if n % 2 != 0:
    median_vol = sortedtrd[n // 2]
else:
    median_vol = (sortedtrd[n // 2 - 1] + sortedtrd[n // 2]) / 2

print("The median trading volume was ${0:.2f} in 2017 ." .format(round(median_vol, 2)))

The median trading volume was $76286.00 in 2017 .
